# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)        
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Estimate Pulse Rate Algorithm

    Estimate Pulse Rate on PGG and acceleration data. 
    Compare with ECG ground truth and caculate MAE and confident

    Returns:
        Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
        np.array(errors): Array contain error for each window 8s
        np.array(confidence): Array contain condidence for each window 8s
        
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Compute pulse rate estimates and estimation confidence.
    # Load BPM
    ref_bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # total time in seconds
    total_time = 5*60
    fs = 125 #len(ppg)/total_time
    
    # if diferent less than threshold_bpm we consider ppg_pulse_rate = acc_dominant_rate
    threshold_bpm = 5
    
    # Use the 40-240BPM range to create your pass band.
    lowcut_fs = 40 / 60
    highcut_fs = 240 / 60
    ppg_bpf = BandpassFilter(ppg, (lowcut_fs,highcut_fs), fs)
    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_mag_bpf = BandpassFilter(acc_mag, (lowcut_fs,highcut_fs), fs)
    
    # caculate number of windows
    window_size = round(8 * fs)
    step_size = round(2 * fs)
    n_windows = round((len(ppg_bpf) - window_size) // step_size + 1)
    if n_windows > len(ref_bpm): 
        n_windows = len(ref_bpm)
        
    errors, confidence = [], []
    # Loop through all windows
    for i in range(n_windows):
        # Get start end of the windows
        start = i * step_size
        end = start + window_size
        # Extract windowed signal
        ppg_window = ppg_bpf[start:end]
        acc_mag_window = acc_mag_bpf[start:end]
        
        #acc_mag_window = np.sqrt(accx_window**2 + accy_window**2 + accz_window**2)
        #acc_mag_window = BandpassFilter(acc_mag_window, (lowcut_fs, highcut_fs), fs)
        
        # Frequency domain analysis
        freqs = np.fft.rfftfreq(len(ppg_window), 1/fs)
        ppg_fft = np.abs(np.fft.rfft(ppg_window))
        acc_mag_fft = np.abs(np.fft.rfft(acc_mag_window))
        
        # PGG Dominant frequencies
        ppg_dominant_freq_idx = np.argmax(ppg_fft)
        ppg_dominant_freq = freqs[ppg_dominant_freq_idx]
        ppg_pulse_rate = ppg_dominant_freq * 60
        
        # Acceleration Dominant frequencies
        acc_dominant_freq_idx = np.argmax(acc_mag_fft)
        acc_dominant_freq = freqs[acc_dominant_freq_idx]
        acc_dominant_rate = acc_dominant_freq * 60
        
        #debug_pgg = np.argsort(ppg_fft)[::-1]
        #print("debug:",i ,ppg_pulse_rate, "shorted", freqs[debug_pgg[1:8]]*60,"acc_dominant_freq",acc_dominant_rate, 'ECG', ref_bpm[i][0])
        
        # Check for interference
        # When the dominant or harmonic accelerometer frequency is the same as the PPG, 
        # try picking the next strongest PPG frequency if there is another good candidate.
        if (np.abs(ppg_pulse_rate - acc_dominant_rate) < threshold_bpm) \
                or (np.abs(ppg_pulse_rate - acc_dominant_rate/2) < threshold_bpm) \
                or (np.abs(ppg_pulse_rate - acc_dominant_rate*2) < threshold_bpm):  # If interference is detected
            sorted_ppg_indices = np.argsort(ppg_fft)[::-1]  # Sort frequencies by power
            # loop through first 4 frequencies
            for idx in sorted_ppg_indices[:4]: 
                candidate_freq = freqs[idx]
                candidate_bpm = candidate_freq * 60
                # Check if any candidate is not the dominant or harmonic accelerometer frequency
                if (np.abs(acc_dominant_rate - candidate_bpm) >= threshold_bpm) \
                        and (np.abs(acc_dominant_rate*2 - candidate_bpm) >= threshold_bpm)\
                        and (np.abs(acc_dominant_rate/2 - candidate_bpm) >= threshold_bpm):
                    ppg_pulse_rate = candidate_bpm
                    break
        
        # Max magnitude frequency
        est_fs = ppg_pulse_rate / 55
        fs_win = 30/60
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        
        conf = np.sum(ppg_fft[fs_win_e])/np.sum(ppg_fft)         
        err = np.abs(ppg_pulse_rate - ref_bpm[i][0])
        
        errors.append(err)
        confidence.append(conf)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.array(errors), np.array(confidence)

def BandpassFilter(signal, pass_band, fs):
    """Bandpass Filter.
    
    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)